# BINARY CLASSIFICATION (ELA)



In [ ]:
import sys, warnings
sys.path.append('..')
warnings.filterwarnings('ignore')

import os, random
from glob import glob 
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from arch import utils, ela, net

In [ ]:
DEVICE = utils.device_mapper()
print(f"Device: {str(DEVICE).upper()}")

---

## DATASET

In [ ]:
# DATASET = "/mnt/artemis/library/datasets/image-forgery"
IF_DATASET = "/mnt/artemis/library/datasets/image-forgery"
CASIA_DATASET = "/mnt/artemis/library/datasets/casia"

In [ ]:
total_files = [file for d in [IF_DATASET, CASIA_DATASET] for file in glob(os.path.join(d, '**', '*'))]
print(f"Total count: {len(total_files)}")

types = set()
for file in total_files: types.add(file.split('.')[-1])
print(f"Types: {types}")

In [ ]:
Au = (
    glob(IF_DATASET + "/Original/*.jpg") + 
    glob(IF_DATASET + "/Original/*.JPG") + 
    glob(IF_DATASET + "/Original/*.tif") + 
    glob(CASIA_DATASET + "/Au/Au*.jpg")
)
Tp = (
    glob(IF_DATASET + "/Forged/*.jpg") + 
    glob(IF_DATASET + "/Forged/*.png") + 
    glob(IF_DATASET + "/Forged/*.tif") + 
    glob(CASIA_DATASET + "/Tp/Tp*.jpg") + 
    glob(CASIA_DATASET + "/Tp/Tp*.tif")
)

print(f"Au files: {len(Au)}")
print(f"Tp files: {len(Tp)}")

Ds = Au + Tp
print(f"Ds files: {len(Ds)}")

In [ ]:
r = 5
c = 25

sampled_paths = random.sample(Ds, r*c)

sample = [Image.open(path).resize((128,128)) for path in sampled_paths[:r*c]]
utils.contact_layer(sample, r, c)

In [ ]:
def split_data(x, p, shuffle=True):
    if shuffle: random.shuffle(Ds)
    bound = int((len(x)/100) * p)
    return x[:bound], x[bound:]

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, transform, filter=None):
        super().__init__()
        self.paths = image_paths
        self.len = len(self.paths)
        self.filter = filter
        
        self.transform = transform

    def __len__(self): return self.len

    def __getitem__(self, idx): 
        path = self.paths[idx]
        x = Image.open(path).convert('RGB')
        if random.random() > 0.5: x = x.transpose(Image.FLIP_LEFT_RIGHT)
        
        if self.filter == 'ela':
            x = ela.compute(x, 90)
            x = self.transform(utils.array2image(x))
        else:
            x = self.transform(x)

        y = 0 if 'Original' in path or 'Au/Au' in path else 1

        return (x, y)

In [ ]:
def denormalize(x, mean=(0.5,), std=(0.5,)):
    mean = torch.tensor(mean).view(-1, 1, 1)
    std = torch.tensor(std).view(-1, 1, 1)
    return x * std + mean

In [ ]:
train, test = split_data(Ds, 95)

Au_count = sum('Original' in path or 'Au' in path for path in train)
Tp_count = sum('Forged' in path or 'Tp' in path for path in train)

print(f"Original: {Au_count} | Forged: {Tp_count}")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
train_ds = CustomDataset(train, transform, filter='ela')
train_dl = DataLoader(train_ds, batch_size=128)
print(f"(Train) Images: {len(train_ds)} | Batches: {len(train_dl)}")

In [ ]:
x, y = next(iter(train_dl))
print(f"Image batch shape: {x.shape}")

r, c = 2, 10
labels = [f"Real [{y[i].item()}]" if y[i].item() == 0 else f"Edited [{y[i].item()}]" for i in range(r*c)]
data = [utils.array2image(denormalize(x[i]).permute(1, 2, 0).numpy()) for i in range(r*c)]

utils.contact_layer(data, r, c, labels, 10)

---

## MODEL

In [ ]:
model = net.CNN()
model.to(DEVICE)

parameters = model.parameters()
print("Nparams:", sum(p.nelement() for p in parameters))
model.eval()

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.001)

epochs = 5
lossi, accui = [], []
for e in range(epochs):
    with tqdm(train_dl, unit='batch') as tepoch:
        for xb, yb in tepoch:
            tepoch.set_description(f"Epoch {e+1}")

            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits = model(xb)
            loss = F.cross_entropy(logits, yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            accuracy = ((logits.argmax(dim=1) == yb).float().mean())
            
            lossi.append(loss.item())
            accui.append(accuracy.item())
            tepoch.set_postfix(loss=loss.item(), accuracy=accuracy.item())

In [ ]:
from datetime import datetime
model_path = f"bce_{datetime.now().strftime('%Y%m%d%H%M%S')}.pth"
torch.save(model.state_dict(), model_path)

In [ ]:
plt.figure(figsize=(25, 5))
plt.plot(lossi, label='loss', lw=0.5)
plt.plot(accui, label='accuracy', lw=0.5)
plt.xlim([0, len(lossi)])
plt.grid(alpha=0.25)
plt.legend(), plt.tight_layout()

---

## TEST / INFER

In [ ]:
model = net.CNN()
model.load_state_dict(torch.load(model_path))
model.to(DEVICE)
print(model.eval())

In [ ]:
test_ds = CustomDataset(test, transform, filter='ela')
test_dl = DataLoader(test_ds, batch_size=128)
print(f"(Test) Images: {len(test_ds)} | Batches: {len(test_dl)}")

In [ ]:
with torch.no_grad():
    with tqdm(test_dl, unit='batch') as tepoch:
        loss, accuracy = 0, 0
        for xt, yt in tepoch:
            tepoch.set_description(f"Testing..")
            xt, yt = xt.to(DEVICE), yt.to(DEVICE)
            logits = model(xt)

            loss += F.cross_entropy(logits, yt)
            accuracy += (logits.argmax(dim=1) == yt).float().mean()

        loss /= len(test_dl)
        accuracy /= len(test_dl)

print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy * 100:.2f}%")

In [ ]:
sample = random.sample(test, 10)
sample_ds = CustomDataset(sample, transform, filter='ela')
sample_dl = DataLoader(sample_ds, batch_size=len(sample_ds))

xt, yt = next(iter(sample_dl))
xt, yt = xt.to(DEVICE), yt.to(DEVICE)
logits = model(xt)
preds = F.softmax(logits, dim=1)[:, 1].tolist()

In [ ]:
_, axs = plt.subplots(2, len(sample_ds), figsize=(len(sample_ds) * 5, 12))
for idx, x in enumerate(xt):
    pred = preds[idx]
    label = "Forged" if pred > 0.5 else "Original"
    axs[0, idx].imshow(Image.open(sample[idx]).resize((224, 224)))
    title_color = "red" if round(pred, 0) != yt[idx] else "black"
    axs[0, idx].set_title(f"{yt[idx].item()} | {pred:.5f} | {label}", color=title_color)
    axs[0, idx].axis(False)
    x = denormalize(x.cpu())
    axs[1, idx].imshow(x.permute(1, 2, 0).numpy())
    axs[1, idx].axis(False)

plt.tight_layout()